# * Prerequisites

You should familiarize yourself with the `numpy.ndarray` class of python's `numpy` library.

You should be able to answer the following questions before starting this assignment. Let's assume `a` is a numpy array.
* What is an array's shape (e.g., what is the meaning of `a.shape`)?  
* What is numpy's reshaping operation? How much computational over-head would it induce?  
* What is numpy's transpose operation, and how it is different from reshaping? Does it cause computation overhead?
* What is the meaning of the commands `a.reshape(-1, 1)` and `a.reshape(-1)`?
* Would happens to the variable `a` after we call `b = a.reshape(-1)`? Does any of `a`'s attributes change?
* How do assignments in python and numpy work in general?
    * Does the `b=a` statement use copying by value? Or is it copying by reference?
    * Is the answer to the previous question change depending on whether `a` is a numpy array or a scalar value?
    
You can answer all of these questions by

    1. Reading numpy's documentation from https://numpy.org/doc/stable/.
    2. Making trials using dummy variables.

# *Assignment Summary

The UC Irvine machine learning data repository hosts a famous collection of data on whether a patient has diabetes (the Pima Indians dataset), originally owned by the National Institute of Diabetes and Digestive and Kidney Diseases and donated by Vincent Sigillito. You can find this data at  https://www.kaggle.com/uciml/pima-indians-diabetes-database/data. This data has a set of attributes of patients, and a categorical variable telling whether the patient is diabetic or not. For several attributes in this data set, a value of 0 may indicate a missing value of the variable.

* **Part 1-A)** Build a simple naive Bayes classifier to classify this data set. We will use 20% of the data for evaluation and the other 80% for training. 

  There are a total of 768 data-points. You should use a normal distribution to model each of the class-conditional distributions. You should write this classifier yourself (it's quite straight-forward).

  Report the accuracy of the classifier on the 20% evaluation data, where accuracy is the number of correct predictions as a fraction of total predictions.

* **Part 1-B)** Now adjust your code so that, for attribute 3 (Diastolic blood pressure), attribute 4 (Triceps skin fold thickness), attribute 6 (Body mass index), and attribute 8 (Age), it regards a value of 0 as a missing value when estimating the class-conditional distributions, and the posterior.

  Report the accuracy of the classifier on the 20% that was held out for evaluation.

* **Part 1-C)** Now install SVMLight, which you can find at http://svmlight.joachims.org, to train and evaluate an SVM to classify this data.

  You don't need to understand much about SVM's to do this as we'll do that in following exercises. You should NOT substitute NA values for zeros for attributes 3, 4, 6, and 8.
  
  Report the accuracy of the classifier on the held out 20%

# 0. Data

## 0.1 Description

The UC Irvine's Machine Learning Data Repository Department hosts a Kaggle Competition with famous collection of data on whether a patient has diabetes (the Pima Indians dataset), originally owned by the National Institute of Diabetes and Digestive and Kidney Diseases and donated by Vincent Sigillito. 

You can find this data at https://www.kaggle.com/uciml/pima-indians-diabetes-database/data. The Kaggle website offers valuable visualizations of the original data dimensions in its dashboard. It is quite insightful to take the time and make sense of the data using their dashboard before applying any method to the data.

## 0.2 Information Summary

* **Input/Output**: This data has a set of attributes of patients, and a categorical variable telling whether the patient is diabetic or not. 

* **Missing Data**: For several attributes in this data set, a value of 0 may indicate a missing value of the variable.

* **Final Goal**: We want to build a classifier that can predict whether a patient has diabetes or not. To do this, we will train multiple kinds of models, and will be handing the missing data with different approaches for each method (i.e., some methods will ignore their existence, while others may do something about the missing data).

## 0.3 Loading

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#from utils import test_case_checker

In [4]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## 0.1 Splitting The Data

First, we will shuffle the data completely, and forget about the order in the original csv file. 

* The training and evaluation dataframes will be named ```train_df``` and ```eval_df```, respectively.

* We will also create the 2-d numpy array `train_features` whose number of rows is the number of training samples, and the number of columns is 8 (i.e., the number of features). We will define `eval_features` in a similar fashion

* We would also create the 1-d numpy arrays `train_labels` and `eval_labels` which contain the training and evaluation labels, respectively.

In [5]:
# Let's generate the split ourselves.
np_random = np.random.RandomState(seed=12345)
rand_unifs = np_random.uniform(0,1,size=df.shape[0])
division_thresh = np.percentile(rand_unifs, 80)
train_indicator = rand_unifs < division_thresh
eval_indicator = rand_unifs >= division_thresh


In [6]:
train_df = df[train_indicator].reset_index(drop=True)
train_features = train_df.loc[:, train_df.columns != 'Outcome'].values
train_labels = train_df['Outcome'].values
train_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [7]:
eval_df = df[eval_indicator].reset_index(drop=True)
eval_features = eval_df.loc[:, eval_df.columns != 'Outcome'].values
eval_labels = eval_df['Outcome'].values
eval_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,3,78,50,32,88,31.0,0.248,26,1
2,10,168,74,0,0,38.0,0.537,34,1
3,0,118,84,47,230,45.8,0.551,31,1
4,7,107,74,0,0,29.6,0.254,31,1


In [8]:
train_features.shape, train_labels.shape, eval_features.shape, eval_labels.shape

((614, 8), (614,), (154, 8), (154,))

## 0.2 Pre-processing The Data

Some of the columns exhibit missing values. We will use a Naive Bayes Classifier later that will treat such missing values in a special way.  To be specific, for attribute 3 (Diastolic blood pressure), attribute 4 (Triceps skin fold thickness), attribute 6 (Body mass index), and attribute 8 (Age), we should regard a value of 0 as a missing value.

Therefore, we will be creating the `train_featues_with_nans` and `eval_features_with_nans` numpy arrays to be just like their `train_features` and `eval_features` counter-parts, but with the zero-values in such columns replaced with nans.

In [9]:
train_df_with_nans = train_df.copy(deep=True)
eval_df_with_nans = eval_df.copy(deep=True)
for col_with_nans in ['BloodPressure', 'SkinThickness', 'BMI', 'Age']:
    train_df_with_nans[col_with_nans] = train_df_with_nans[col_with_nans].replace(0, np.nan)
    eval_df_with_nans[col_with_nans] = eval_df_with_nans[col_with_nans].replace(0, np.nan)
train_features_with_nans = train_df_with_nans.loc[:, train_df_with_nans.columns != 'Outcome'].values
eval_features_with_nans = eval_df_with_nans.loc[:, eval_df_with_nans.columns != 'Outcome'].values

In [10]:
print('Here are the training rows with at least one missing values.')
print('')
print('You can see that such incomplete data points constitute a substantial part of the data.')
print('')
nan_training_data = train_df_with_nans[train_df_with_nans.isna().any(axis=1)]
nan_training_data

Here are the training rows with at least one missing values.

You can see that such incomplete data points constitute a substantial part of the data.



,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
1,8,183,64.0,NaN,0,23.3,0.672,32,1
4,5,116,74.0,NaN,0,25.6,0.201,30,0
5,10,115,NaN,NaN,0,35.3,0.134,29,0
7,8,125,96.0,NaN,0,NaN,0.232,54,1
8,4,110,92.0,NaN,0,37.6,0.191,30,0
...,...,...,...,...,...,...,...,...,...
598,6,162,62.0,NaN,0,24.3,0.178,50,1
599,4,136,70.0,NaN,0,31.2,1.182,22,1
605,1,106,76.0,NaN,0,37.5,0.197,26,0
606,6,190,92.0,NaN,0,35.5,0.278,66,1


# 1. Part 1 (Building a simple Naive Bayes Classifier)

Consider a single sample $(\mathbf{x}, y)$, where the feature vector is denoted with $\mathbf{x}$, and the label is denoted with $y$. We will also denote the $j^{th}$ feature of $\mathbf{x}$ with $x^{(j)}$.

According to the textbook, the Naive Bayes Classifier uses the following decision rule:

"Choose $y$ such that $$\bigg[\log p(y) + \sum_{j} \log p(x^{(j)}|y) \bigg]$$ is the largest"

However, we first need to define the probabilistic models of the prior $p(y)$ and the class-conditional feature distributions $p(x^{(j)}|y)$ using the training data.

* **Modelling the prior $p(y)$**: We fit a Bernoulli distribution to the `Outcome` variable of `train_df`.
* **Modelling the class-conditional feature distributions $p(x^{(j)}|y)$**: We fit Gaussian distributions, and infer the Gaussian mean and variance parameters from `train_df`.

# <span style="color:blue">Task 1</span>

Write a function `log_prior` that takes a numpy array `train_labels` as input, and outputs the following vector as a column numpy array (i.e., with shape $(2,1)$).

$$\log p_y =\begin{bmatrix}\log p(y=0)\\\log p(y=1)\end{bmatrix}$$

Try and avoid the utilization of loops as much as possible. No loops are necessary.

**Hint**: Make sure all the array shapes are what you need and expect. You can reshape any numpy array without any tangible computational over-head.

In [11]:
def log_prior(train_labels):
    ### BEGIN SOLUTION
    py_1=train_labels.mean()
    py_0=1-py_1
    log_py=np.array([np.log(py_0),np.log(py_1)]).reshape(2,1)
    ### END SOLUTION
    assert log_py.shape == (2,1)
    return log_py

In [12]:
log_py = log_prior(train_labels)
log_py

array([[-0.41610786],
       [-1.07766068]])

# <span style="color:blue">Task 2</span>

Write a function `cc_mean_ignore_missing` that takes the numpy arrays `train_features` and `train_labels` as input, and outputs the following matrix with the shape $(8,2)$, where 8 is the number of features.

$$\mu_y = \begin{bmatrix} \mathbb{E}[x^{(0)}|y=0] & \mathbb{E}[x^{(0)}|y=1]\\
\mathbb{E}[x^{(1)}|y=0] & \mathbb{E}[x^{(1)}|y=1] \\
\cdots & \cdots\\
\mathbb{E}[x^{(7)}|y=0] & \mathbb{E}[x^{(7)}|y=1]\end{bmatrix}$$

Some points regarding this task:

* The `train_features` numpy array has a shape of `(N,8)` where `N` is the number of training data points, and 8 is the number of the features. 

* The `train_labels` numpy array has a shape of `(N,)`. 

* **You can assume that `train_features` has no missing elements in this task**.

* Try and avoid the utilization of loops as much as possible. No loops are necessary.

In [13]:
def cc_mean_ignore_missing(train_features, train_labels):
    N, d = train_features.shape
    ### BEGIN SOLUTION
    means_success = np.mean(train_features[train_labels==1],axis=0)
    means_failure = np.mean(train_features[train_labels==0],axis=0)
    mu_y = np.column_stack((means_failure,means_success))
    ### END SOLUTION
    assert mu_y.shape == (d, 2)
    return mu_y

In [14]:
mu_y = cc_mean_ignore_missing(train_features, train_labels)
mu_y

array([[  3.48641975,   4.91866029],
       [109.99753086, 142.30143541],
       [ 68.77037037,  70.66028708],
       [ 19.51358025,  21.97129187],
       [ 66.25679012, 100.55980861],
       [ 30.31703704,  35.1492823 ],
       [  0.42825926,   0.55279904],
       [ 31.57283951,  37.39712919]])

# <span style="color:blue">Task 3</span>

Write a function `cc_std_ignore_missing` that takes the numpy arrays `train_features` and `train_labels` as input, and outputs the following matrix with the shape $(8,2)$, where 8 is the number of features.

$$\sigma_y = \begin{bmatrix} \text{std}[x^{(0)}|y=0] & \text{std}[x^{(0)}|y=1]\\
\text{std}[x^{(1)}|y=0] & \text{std}[x^{(1)}|y=1] \\
\cdots & \cdots\\
\text{std}[x^{(7)}|y=0] & \text{std}[x^{(7)}|y=1]\end{bmatrix}$$

Some points regarding this task:

* The `train_features` numpy array has a shape of `(N,8)` where `N` is the number of training data points, and 8 is the number of the features. 

* The `train_labels` numpy array has a shape of `(N,)`. 

* **You can assume that `train_features` has no missing elements in this task**.

* Try and avoid the utilization of loops as much as possible. No loops are necessary.

In [15]:
def cc_std_ignore_missing(train_features, train_labels):
    N, d = train_features.shape
    ### BEGIN SOLUTION
    sigma_success = np.std(train_features[train_labels==1],axis=0)
    sigma_failure = np.std(train_features[train_labels==0],axis=0)
    sigma_y = np.column_stack((sigma_failure,sigma_success))
    ### END SOLUTION
    assert sigma_y.shape == (d, 2)
    return sigma_y

In [16]:
sigma_y = cc_std_ignore_missing(train_features, train_labels)
sigma_y

array([[  3.1155426 ,   3.75417931],
       [ 25.96811899,  32.50910874],
       [ 18.07540068,  21.69568568],
       [ 15.02320635,  17.21685884],
       [ 95.63339586, 139.24364214],
       [  7.50030986,   6.6625219 ],
       [  0.29438217,   0.37201494],
       [ 11.67577435,  11.01543899]])

# <span style="color:blue">Task 4</span>

Write a function `log_prob` that takes the numpy arrays `train_features`, $\mu_y$, $\sigma_y$, and  $\log p_y$ as input, and outputs the following matrix with the shape $(N, 2)$

$$\log p_{x,y} = \begin{bmatrix} \bigg[\log p(y=0) + \sum_{j=0}^{7} \log p(x_1^{(j)}|y=0) \bigg] & \bigg[\log p(y=1) + \sum_{j=0}^{7} \log p(x_1^{(j)}|y=1) \bigg] \\
\bigg[\log p(y=0) + \sum_{j=0}^{7} \log p(x_2^{(j)}|y=0) \bigg] & \bigg[\log p(y=1) + \sum_{j=0}^{7} \log p(x_2^{(j)}|y=1) \bigg] \\
\cdots & \cdots \\
\bigg[\log p(y=0) + \sum_{j=0}^{7} \log p(x_N^{(j)}|y=0) \bigg] & \bigg[\log p(y=1) + \sum_{j=0}^{7} \log p(x_N^{(j)}|y=1) \bigg] \\
\end{bmatrix}$$

where
* $N$ is the number of training data points.
* $x_i$ is the $i^{th}$ training data point.

Try and avoid the utilization of loops as much as possible. No loops are necessary.

**Hint**: Remember that we are modelling $p(x_i^{(j)}|y)$ with a Gaussian whose parameters are defined inside $\mu_y$ and $\sigma_y$. Write the Gaussian PDF expression and take its natural log **on paper**, then implement it.

**Important Note**: Do not use third-party and non-standard implementations for computing $\log p(x_i^{(j)}|y)$. Using functions that find the Gaussian PDF, and then taking their log is **numerically unstable**; the Gaussian PDF values can easily become extremely small numbers that cannot be represented using floating point standards and thus would be stored as zero. Taking the log of a zero value will throw an error. On the other hand, it is unnecessary to compute and store $p(x_i^{(j)}|y)$ in order to find $\log p(x_i^{(j)}|y)$; you can write $\log p(x_i^{(j)}|y)$ as a direct function of $\mu_y$, $\sigma_y$ and the features. This latter approach is numerically stable, and can be applied when the PDF values are much smaller than could be stored using the common standards.

In [17]:
def log_prob(features, mu_y, sigma_y, log_py):
    N, d = features.shape
    ### BEGIN SOLUTION
    log_sigma_inv = np.log((1/np.sqrt(2*np.pi))*(1./sigma_y))
    x_mu_sigma_false = features - mu_y[:,0].reshape(1,-1)
    x_mu_sigma_false = x_mu_sigma_false / sigma_y[:,0].reshape(1,-1)
    x_mu_sigma_false = (-1./2.)*np.square(x_mu_sigma_false)
    log_p_x_false = log_sigma_inv[:,0].reshape(1,-1) + x_mu_sigma_false
    log_p_x_false = np.sum(log_p_x_false, axis = 1, keepdims = True) 
    log_p_x_false = log_py[0] +log_p_x_false

    x_mu_sigma_true = features - mu_y[:,1].reshape(1,-1)
    x_mu_sigma_true = x_mu_sigma_true / sigma_y[:,1].reshape(1,-1)
    x_mu_sigma_true = (-1./2.)*np.square(x_mu_sigma_true)
    log_p_x_true = log_sigma_inv[:,1].reshape(1,-1) + x_mu_sigma_true
    log_p_x_true = np.sum(log_p_x_true, axis = 1, keepdims = True) 
    log_p_x_true = log_py[1] +log_p_x_true
    
    log_p_x_y = np.concatenate((log_p_x_false, log_p_x_true), axis = 1)
    ### END SOLUTION    
    assert log_p_x_y.shape == (N,2)
    return log_p_x_y

In [18]:
log_p_x_y = log_prob(train_features, mu_y, sigma_y, log_py)
log_p_x_y

array([[-26.96647828, -31.00418408],
       [-32.4755447 , -31.39530914],
       [-27.14875996, -31.51999532],
       ...,
       [-26.29368771, -29.09161966],
       [-28.19432943, -30.08324788],
       [-26.98605248, -30.80571318]])

## 1.1. Writing the Simple Naive Bayes Classifier

In [19]:
class NBClassifier():
    def __init__(self, train_features, train_labels):
        self.train_features = train_features
        self.train_labels = train_labels
        self.log_py = log_prior(train_labels)
        self.mu_y = self.get_cc_means()
        self.sigma_y = self.get_cc_std()
        
    def get_cc_means(self):
        mu_y = cc_mean_ignore_missing(self.train_features, self.train_labels)
        return mu_y
    
    def get_cc_std(self):
        sigma_y = cc_std_ignore_missing(self.train_features, self.train_labels)
        return sigma_y
    
    def predict(self, features):
        log_p_x_y = log_prob(features, mu_y, sigma_y, log_py)
        return log_p_x_y.argmax(axis=1)

In [20]:
diabetes_classifier = NBClassifier(train_features, train_labels)
train_pred = diabetes_classifier.predict(train_features)
eval_pred = diabetes_classifier.predict(eval_features)

In [21]:
train_acc = (train_pred==train_labels).mean()
eval_acc = (eval_pred==eval_labels).mean()
print(f'The training data accuracy of your trained model is {train_acc}')
print(f'The evaluation data accuracy of your trained model is {eval_acc}')

The training data accuracy of your trained model is 0.7671009771986971
The evaluation data accuracy of your trained model is 0.7532467532467533


## 1.2 Running an off-the-shelf implementation of Naive-Bayes For Comparison

In [22]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB().fit(train_features, train_labels)
train_pred_sk = gnb.predict(train_features)
eval_pred_sk = gnb.predict(eval_features)
print(f'The training data accuracy of your trained model is {(train_pred_sk == train_labels).mean()}')
print(f'The evaluation data accuracy of your trained model is {(eval_pred_sk == eval_labels).mean()}')

The training data accuracy of your trained model is 0.7671009771986971
The evaluation data accuracy of your trained model is 0.7532467532467533


# Part 2 (Building a Naive Bayes Classifier Considering Missing Entries)

In this part, we will modify some of the parameter inference functions of the Naive Bayes classifier to make it able to ignore the NaN entries when inferring the Gaussian mean and stds.

# <span style="color:blue">Task 5</span>

Write a function `cc_mean_consider_missing` that
* has exactly the same input and output types as the `cc_mean_ignore_missing` function,
* and has similar functionality to `cc_mean_ignore_missing` except that it can handle and ignore the NaN entries when computing the class conditional means.

You can borrow most of the code from your `cc_mean_ignore_missing` implementation, but you should make it compatible with the existence of NaN values in the features.

Try and avoid the utilization of loops as much as possible. No loops are necessary.

* **Hint**: You may find the `np.nanmean` function useful.

In [23]:
def cc_mean_consider_missing(train_features_with_nans, train_labels):
    N, d = train_features_with_nans.shape
    
    ### BEGIN SOLUTION
    means_success = np.nanmean(train_features_with_nans[train_labels==1],axis=0)
    means_failure = np.nanmean(train_features_with_nans[train_labels==0],axis=0)
    mu_y = np.column_stack((means_failure,means_success))
    ### END SOLUTION

    
    assert not np.isnan(mu_y).any()
    assert mu_y.shape == (d, 2)
    return mu_y

In [24]:
mu_y = cc_mean_consider_missing(train_features_with_nans, train_labels)
mu_y

array([[  3.48641975,   4.91866029],
       [109.99753086, 142.30143541],
       [ 71.41538462,  75.34693878],
       [ 27.53658537,  32.11188811],
       [ 66.25679012, 100.55980861],
       [ 30.85025126,  35.31826923],
       [  0.42825926,   0.55279904],
       [ 31.57283951,  37.39712919]])

# <span style="color:blue">Task 6</span>

Write a function `cc_std_consider_missing` that
* has exactly the same input and output types as the `cc_std_ignore_missing` function,
* and has similar functionality to `cc_std_ignore_missing` except that it can handle and ignore the NaN entries when computing the class conditional means.

You can borrow most of the code from your `cc_std_ignore_missing` implementation, but you should make it compatible with the existence of NaN values in the features.

Try and avoid the utilization of loops as much as possible. No loops are necessary.

* **Hint**: You may find the `np.nanstd` function useful.

In [25]:
def cc_std_consider_missing(train_features_with_nans, train_labels):
    N, d = train_features_with_nans.shape
    
    ### BEGIN SOLUTION
    sigma_success = np.nanstd(train_features_with_nans[train_labels==1],axis=0)
    sigma_failure = np.nanstd(train_features_with_nans[train_labels==0],axis=0)
    sigma_y = np.column_stack((sigma_failure,sigma_success))
    ### END SOLUTION
     
    assert not np.isnan(sigma_y).any()
    assert sigma_y.shape == (d, 2)
    return sigma_y

In [26]:
sigma_y = cc_std_consider_missing(train_features_with_nans, train_labels)
sigma_y

array([[  3.1155426 ,   3.75417931],
       [ 25.96811899,  32.50910874],
       [ 12.26342359,  12.1982786 ],
       [  9.87753687,  10.37284304],
       [ 95.63339586, 139.24364214],
       [  6.38703834,   6.21564813],
       [  0.29438217,   0.37201494],
       [ 11.67577435,  11.01543899]])

## 2.1. Writing the Naive Bayes Classifier With Missing Data Handling

In [27]:
class NBClassifierWithMissing(NBClassifier):
    def get_cc_means(self):
        mu_y = cc_mean_consider_missing(self.train_features, self.train_labels)
        return mu_y
    
    def get_cc_std(self):
        sigma_y = cc_std_consider_missing(self.train_features, self.train_labels)
        return sigma_y

In [28]:
diabetes_classifier_nans = NBClassifierWithMissing(train_features_with_nans, train_labels)
train_pred = diabetes_classifier_nans.predict(train_features_with_nans)
eval_pred = diabetes_classifier_nans.predict(eval_features_with_nans)

In [29]:
train_acc = (train_pred==train_labels).mean()
eval_acc = (eval_pred==eval_labels).mean()
print(f'The training data accuracy of your trained model is {train_acc}')
print(f'The evaluation data accuracy of your trained model is {eval_acc}')

The training data accuracy of your trained model is 0.7182410423452769
The evaluation data accuracy of your trained model is 0.7142857142857143


# 3. Running SVMlight

In this section, we are going to investigate the support vector machine classification method. We will become familiar with this classification method in week 3. However, in this section, we are just going to observe how this method performs to set the stage for the third week.

`SVMlight` (http://svmlight.joachims.org/) is a famous implementation of the SVM classifier. 

`SVMLight` can be called from a shell terminal, and there is no nice wrapper for it in python3. Therefore:
1. We have to export the training data to a special format called `svmlight/libsvm`. This can be done using scikit-learn.
2. We have to run the `svm_learn` program to learn the model and then store it.
3. We have to import the model back to python.

## 3.1 Exporting the training data to libsvm format

In [37]:
from scikit_learn.sklearn.datasets import dump_svmlight_file
dump_svmlight_file(train_features, 2*train_labels-1, 'training_feats.data', 
                   zero_based=False, comment=None, query_id=None, multilabel=False)

ModuleNotFoundError: ignored

## 3.2 Training `SVMlight`

In [67]:
!chmod 755 -R svm_learn
from subprocess import Popen, PIPE
process = Popen(["./svm_learn", "/content/training_feats.data", "/content/svm_model.txt"], stdout=PIPE, stderr=PIPE)
stdout, stderr = process.communicate()
print(stdout.decode("utf-8"))

Scanning examples...done
Reading examples into memory...100..200..300..400..500..600..OK. (614 examples read)
Setting default regularization parameter C=0.0000
Optimizing..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## 3.3 Importing the SVM Model

In [68]:
from svm2weight import get_svmlight_weights
svm_weights, thresh = get_svmlight_weights('svm_model.txt', printOutput=False)

def svmlight_classifier(train_features):
    return (train_features @ svm_weights - thresh).reshape(-1) >= 0.

In [69]:
train_pred = svmlight_classifier(train_features)
eval_pred = svmlight_classifier(eval_features)

In [70]:
train_acc = (train_pred==train_labels).mean()
eval_acc = (eval_pred==eval_labels).mean()
print(f'The training data accuracy of your trained model is {train_acc}')
print(f'The evaluation data accuracy of your trained model is {eval_acc}')

The training data accuracy of your trained model is 0.7703583061889251
The evaluation data accuracy of your trained model is 0.7402597402597403
